In [53]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct
from qdrant_client.http import models

from qdrant_client import QdrantClient
from qdrant_client.http.models import Filter, FieldCondition, Range, PointIdsList, SearchParams, Vector

import sqlite3
import pandas as pd

In [3]:
client = QdrantClient(host="sprakbankdb1.lx.nb.no", port=6333)

In [4]:
import numpy as np

In [5]:
collections = client.http.collections_api.get_collections().result.collections

# Print the names of all collections
for collection in collections:
    print(collection.name)

vss_1850_cos
vss_dewey_code
vss_1800_cos
vss_dewey
images_1900_cos


In [6]:
collection_name = "vss_1850_cos"

In [7]:
#db = "../Embeddings/vss_1850.db"

In [12]:
def query(db, sql, params=()):
    with sqlite3.connect(db) as con:
        r = con.execute(sql, params).fetchall()
    return r

In [13]:
def convert_vector(binary_vector):
    # Assuming binary_vector is a blob from SQLite, convert it to a list
    return np.frombuffer(binary_vector, dtype=np.float32).tolist()

In [36]:
def search_word(word, n=20):
    res = query(db, "select * from words where word = ?", (word,))
    
    query_vector = convert_vector(res[0][1])
    
    hits = client.search(
       collection_name = collection_name,
       query_vector = query_vector,
       limit = n  # 
    )
    
    
    return pd.DataFrame([(x.payload['word'], x.score) for x in hits], columns=["Word", "Score"])
    

In [37]:
search_word('Eskimoerne', 8)

,Word,Score
0,Eskimoerne,1.000000
1,Kafferne,0.860801
2,Negerne,0.846766
3,Grønlænderne,0.838771
4,Hottentotterne,0.827435
5,Indianerne,0.822096
6,Europæerne,0.819458
7,Malayerne,0.814286


In [24]:
try:
    count_response = client.count(collection_name=collection_name)
    total_points = count_response.count
    print(f"Total points in collection '{collection_name}': {total_points}")
except Exception as e:
    print(f"An error occurred: {e}")

Total points in collection 'vss_1850_cos': 2168213


In [54]:
# The word you're searching for
search_word = 'Eskimoerne'

def search_word(client, search_word, limit=10):
    """Search a word using a client"""
    
   
    response = client.scroll(
    
        scroll_filter=models.Filter(
            must=[
                models.FieldCondition(key="word", match=models.MatchValue(value=search_word)),
            ]
        ),
        limit=1,
        with_payload=True,
        with_vectors=True,
    
        collection_name=collection_name,
        
    )
    r = response[0][0]
    #print(r)
    # Check if the word exists in the collection and fetch its vector
    if r:
        word_vector = r.vector
    
        # Step 2: Perform a similarity search with the fetched vector
        similar_words_response = client.search(
            collection_name=collection_name,
            query_vector=word_vector,
            #search_params=SearchParams(k=10),  # Adjust 'k' to change the number of similar words to find
            limit=limit  # Adjust limit to change the number of results returned
        )
    
        # Print the similar words found
        #print(similar_words_response)
        
        
        result = [(hit.payload['word'], hit.score) for hit in similar_words_response]
    else:
        result = []
    return result

In [56]:
search_word(client, "Demokrati", limit=30)

[('Demokrati', 1.0000001),
 ('Bureaukrati', 0.89253783),
 ('Aristokrati', 0.834911),
 ('Hierarchi', 0.81994385),
 ('Despoti', 0.8033606),
 ('Monarki', 0.79690707),
 ('Hierarki', 0.79461986),
 ('Regjeringssystem', 0.7841758),
 ('Enevælde', 0.78321457),
 ('Styrelsessæt', 0.7818777),
 ('Monarchi', 0.77367705),
 ('Statssamfund', 0.7695782),
 ('Aristokratie', 0.7672338),
 ('Overhus', 0.7671656),
 ('demokrati', 0.76078975),
 ('Kastevæsen', 0.76034355),
 ('Statsvæsen', 0.75871664),
 ('Flertalsstyre', 0.7576455),
 ('Retsbegreb', 0.75659114),
 ('Selvstyre', 0.75143546),
 ('Hedenskab', 0.75054014),
 ('Pavedømme', 0.74890924),
 ('Statsliv', 0.7487499),
 ('Dogme', 0.74860376),
 ('Theokrati', 0.7463454),
 ('Frimureri', 0.7401727),
 ('Gudsbegreb', 0.7390011),
 ('Statssystem', 0.7362392),
 ('Regeringssystem', 0.7347422),
 ('Oligarchi', 0.7336982)]